In [24]:
import pandas as pd
from sodapy import Socrata
from config import API_Key, username, password
import requests
import json

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.io import show, output_notebook
from bokeh.transform import dodge
from bokeh.core.properties import value

In [22]:
# Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofnewyork.us", API_Key, username, password)

# Returned as JSON from API by sodapy.
data = client.get("43nn-pn8j",limit=381912)

# Convert to pandas DataFrame
data_df = pd.DataFrame.from_records(data)

In [19]:
data_df

,action,boro,building,camis,critical_flag,cuisine_description,dba,grade,grade_date,inspection_date,inspection_type,phone,record_date,score,street,violation_code,violation_description,zipcode
0,Violations were cited in the following area(s).,BRONX,1007,30075445,Not Critical,Bakery,MORRIS PARK BAKE SHOP,A,2018-05-11T00:00:00.000,2018-05-11T00:00:00.000,Cycle Inspection / Initial Inspection,7188924968,2019-04-29T06:00:53.000,5,MORRIS PARK AVE,08C,Pesticide use not in accordance with label or ...,10462
1,Violations were cited in the following area(s).,BRONX,1007,30075445,Not Critical,Bakery,MORRIS PARK BAKE SHOP,A,2018-05-11T00:00:00.000,2018-05-11T00:00:00.000,Cycle Inspection / Initial Inspection,7188924968,2019-04-29T06:00:53.000,5,MORRIS PARK AVE,10F,Non-food contact surface improperly constructe...,10462
2,Violations were cited in the following area(s).,BRONX,1007,30075445,Critical,Bakery,MORRIS PARK BAKE SHOP,A,2017-05-18T00:00:00.000,2017-05-18T00:00:00.000,Cycle Inspection / Initial Inspection,7188924968,2019-04-29T06:00:53.000,7,MORRIS PARK AVE,06D,"Food contact surface not properly washed, rins...",10462
3,Violations were cited in the following area(s).,BRONX,1007,30075445,Not Critical,Bakery,MORRIS PARK BAKE SHOP,A,2017-05-18T00:00:00.000,2017-05-18T00:00:00.000,Cycle Inspection / Initial Inspection,7188924968,2019-04-29T06:00:53.000,7,MORRIS PARK AVE,10F,Non-food contact surface improperly constructe...,10462
4,Violations were cited in the following area(s).,BRONX,1007,30075445,Critical,Bakery,MORRIS PARK BAKE SHOP,A,2016-02-18T00:00:00.000,2016-02-18T00:00:00.000,Cycle Inspection / Initial Inspection,7188924968,2019-04-29T06:00:53.000,10,MORRIS PARK AVE,04L,Evidence of mice or live mice present in facil...,10462
5,Violations were cited in the following area(s).,BRONX,1007,30075445,Not Critical,Bakery,MORRIS PARK BAKE SHOP,A,2016-02-18T00:00:00.000,2016-02-18T00:00:00.000,Cycle Inspection / Initial Inspection,7188924968,2019-04-29T06:00:53.000,10,MORRIS PARK AVE,08A,Facility not vermin proof. Harborage or condit...,10462
6,Violations were cited in the following area(s).,BROOKLYN,469,30112340,Critical,Hamburgers,WENDY'S,A,2019-03-04T00:00:00.000,2019-03-04T00:00:00.000,Cycle Inspection / Initial Inspection,7182875005,2019-04-29T06:00:53.000,13,FLATBUSH AVENUE,02G,Cold food item held above 41Âº F (smoked fish ...,11225
7,Violations were cited in the following area(s).,BROOKLYN,469,30112340,Not Critical,Hamburgers,WENDY'S,A,2019-03-04T00:00:00.000,2019-03-04T00:00:00.000,Cycle Inspection / Initial Inspection,7182875005,2019-04-29T06:00:53.000,13,FLATBUSH AVENUE,10B,Plumbing not properly installed or maintained;...,11225
8,Violations were cited in the following area(s).,BROOKLYN,469,30112340,Not Critical,Hamburgers,WENDY'S,A,2019-03-04T00:00:00.000,2019-03-04T00:00:00.000,Cycle Inspection / Initial Inspection,7182875005,2019-04-29T06:00:53.000,13,FLATBUSH AVENUE,10F,Non-food contact surface improperly constructe...,11225
9,Violations were cited in the following area(s).,BROOKLYN,469,30112340,Critical,Hamburgers,WENDY'S,A,2018-03-13T00:00:00.000,2018-03-13T00:00:00.000,Cycle Inspection / Initial Inspection,7182875005,2019-04-29T06:00:53.000,12,FLATBUSH AVENUE,04L,Evidence of mice or live mice present in facil...,11225


In [23]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381912 entries, 0 to 381911
Data columns (total 18 columns):
action                   380609 non-null object
boro                     381912 non-null object
building                 381689 non-null object
camis                    381912 non-null object
critical_flag            381912 non-null object
cuisine_description      381912 non-null object
dba                      381403 non-null object
grade                    192351 non-null object
grade_date               189820 non-null object
inspection_date          381912 non-null object
inspection_type          380609 non-null object
phone                    381903 non-null object
record_date              381912 non-null object
score                    363727 non-null object
street                   381912 non-null object
violation_code           376126 non-null object
violation_description    373524 non-null object
zipcode                  376362 non-null object
dtypes: object(18)
memory